# Read Excel Documents

Read MS Excel Documents with Python and the `openpyxl` library, and extract tables. The Excel was created with merged cells to make it a little bit more complex to process.

### Install the Required Libraries

In [ ]:
%pip install openpyxl pandas


### Python Imports

In [2]:
import sys
sys.path.append('..\\code')

import os
from dotenv import load_dotenv
load_dotenv()

from docx import Document
import pandas as pd
from docx.shared import Inches
from docx.image.image import Image

from IPython.display import display, Markdown, HTML
from PIL import Image
from doc_utils import *
from utils.bcolors import bcolors as bc  


def show_img(img_path, width = None):
    if width is not None:
        display(HTML(f'<img src="{img_path}" width={width}>'))
    else:
        display(Image.open(img_path))

### Make sure we have the OpenAI Models information

We will need the GPT-4-Turbo and GPT-4-Vision models for this notebook.

When running the below cell, the values should reflect the OpenAI reource you have created in 

In [ ]:
model_info = {
        'AZURE_OPENAI_RESOURCE': os.environ.get('AZURE_OPENAI_RESOURCE'),
        'AZURE_OPENAI_KEY': os.environ.get('AZURE_OPENAI_KEY'),
        'AZURE_OPENAI_MODEL_VISION': os.environ.get('AZURE_OPENAI_MODEL_VISION'),
        'AZURE_OPENAI_MODEL': os.environ.get('AZURE_OPENAI_MODEL'),
}

model_info


### Code Definitions

Defining the function that will read in the docx file and return the text, images and tables as a list of strings, list of images and list of pandas dataframes respectively.

In [18]:

def get_excel_sheet_names(file_path):
    """
    Returns a list of sheet names from the specified Excel file.

    :param file_path: Path to the Excel file
    :return: List of sheet names
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    # Get the list of sheet names
    sheet_names = xls.sheet_names

    return sheet_names


def read_excel_to_dataframes(file_path):
    """
    Reads an Excel file and returns a dictionary of DataFrames.
    Each key in the dictionary corresponds to a sheet name,
    and each value is a DataFrame containing the data from that sheet.

    :param file_path: Path to the Excel file
    :return: Dictionary of DataFrames
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    # Dictionary to hold DataFrames
    dfs = {}

    # Read each sheet into a DataFrame
    for sheet_name in xls.sheet_names:
        dfs[sheet_name] = pd.read_excel(xls, sheet_name, header=None)
        # dfs[sheet_name] = dfs[sheet_name].ffill(axis=1)
        # dfs[sheet_name] = dfs[sheet_name].ffill(axis=0)

    return dfs


### Read Excel

Read the excel and print the dataframes out.

In [19]:
# Usage
file_path = 'sample_data/1_sample_trivia.xlsx'  
dataframes = read_excel_to_dataframes(file_path)

for sheet, df in dataframes.items():
    display(df)


,0,1,2
0,NaN,Trivia QnA Round # 1,NaN
1,Category,Question 1,Answer 1
2,Science,What is the chemical symbol for gold?,Au
3,NaN,How many planets are in the Solar System?,8
4,Geography,What is the capital of Japan?,Tokyo
5,NaN,What is the largest desert in the world?,Sahara
6,History,In what year did the Titanic sink?,1912
7,NaN,Who was the first woman to fly solo across the...,Amelia Earhart
8,NaN,Trivia QnA Round # 2,NaN
9,Category,Question 2,Answer 2


### Print the Tables as Markdown

In [6]:
for sheet, df in dataframes.items():
    display(Markdown(df.to_markdown()))


|    | 0         | 1                                                        | 2                    |
|---:|:----------|:---------------------------------------------------------|:---------------------|
|  0 | nan       | Trivia QnA Round # 1                                     | Trivia QnA Round # 1 |
|  1 | Category  | Question 1                                               | Answer 1             |
|  2 | Science   | What is the chemical symbol for gold?                    | Au                   |
|  3 | Science   | How many planets are in the Solar System?                | 8                    |
|  4 | Geography | What is the capital of Japan?                            | Tokyo                |
|  5 | Geography | What is the largest desert in the world?                 | Sahara               |
|  6 | History   | In what year did the Titanic sink?                       | 1912                 |
|  7 | History   | Who was the first woman to fly solo across the Atlantic? | Amelia Earhart       |
|  8 | History   | Trivia QnA Round # 2                                     | Trivia QnA Round # 2 |
|  9 | Category  | Question 2                                               | Answer 2             |
| 10 | Science   | What gas do plants absorb from the atmosphere?           | Carbon Dioxide       |
| 11 | Science   | How long does Earth take to orbit the Sun?               | 365.25 days          |
| 12 | Geography | What country has the longest coastline?                  | Canada               |
| 13 | Geography | Largest country by area                                  | Russia               |
| 14 | History   | Who invented the lightbulb?                              | Thomas Edison        |
| 15 | History   | The war of 1812 was between which two countries?         | USA and UK           |

### Query the Table

Local RAG demo with the table extracted from the excel sheet.

In [11]:
prompt = """

## START OF TABLE
{markdown}
## END OF TABLE

{query}

"""

def query_table(query):
    p = prompt.format(markdown = dataframes['Sheet1'], query = query)
    output = ask_LLM(p, model_info=model_info)
    print(output)

query_table('how many rounds does this trivia game have?')

The trivia game has two rounds, as indicated by the headers "Trivia QnA Round # 1" and "Trivia QnA Round # 2" in the table.


In [12]:
query_table('Accoring to the trivia game, What is Canada known for?')

According to the trivia game, Canada is known for having the longest coastline.


In [13]:
query_table('How many trivia categories are used in the trivia game?')

There are three trivia categories used in the trivia game:

1. Science
2. Geography
3. History
